In [ ]:
### Only run this code if necessary to install packages 
### on a new anaconda installation. 
### (Remove the """ before and after to uncomment.)

"""
!conda update conda --yes
!conda install -c conda-forge trackpy --yes
!pip install pims
!pip install moviepy
!pip3 install opencv-python  
# https://stackoverflow.com/questions/46610689/how-to-import-cv2-in-python3
""";

In [ ]:
#%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import pims
import trackpy as tp
import os
import time

%pylab inline

from __future__ import division  # this makes mathematical division work better


In [ ]:
"""
Three cells for setting up where all the files are, which objective you used, 
and whether you want to background the images (if the illumination is uneven or if there is dust on the camera).
"""
myhome = r'C:\Users\vhorowit\Documents'
datafolder = r'Research 2021\Data'
data_date = '2021-10-20'
#movienumber = '02'
subfolder = 'PEG 200'
moviename = 'WPC, 15fps, PEG 200_0'
## either generate or manually set the filename with the appropriate path
filename = os.path.join(myhome, datafolder, data_date, subfolder, moviename + '.avi')

using_darkcount = False

## give filename if you are using_darkcount
darkcount_filename = os.path.join(myhome, datafolder, data_date, subfolder, 'darkcount.avi')

filename

In [ ]:
# REMEMBER: you need to write down the objective when you take data!
# scaling, measured in microns per pixel (information about the objective)
scaling = 330 / 1247.96 # 20x1.0, measured 2021-06-17
#scaling = 220 / 1250.04 # 20x1.5, measured 2021-06-17
#scaling = 150 / 1127.54 # 40x1.0, measured 2021-06-16
#scaling = 100 / 1130.61 # 40x1.5, measured 2021-06-16
#scaling = 80 / 914.92 # 60x1.0, measured 2021-05-28
#scaling = 60 / 1031.07 # 60x1.5, measured 2021-05-28

fps = 15.0              # REMEMBER: you need to write this down when you take data!
frametime = 1000/fps    # milliseconds

# set to True if you don't want to background at all. False means it still needs to do the backgrounding.
bg_flag = True;


## The rest of this cell doesn't need to be updated, so just go to the next cell.

## turn video to grey, source: https://soft-matter.github.io/pims/dev/pipelines.html
@pims.pipeline
def as_grey(frame):
    red = frame[:, :, 0]
    green = frame[:, :, 1]
    blue = frame[:, :, 2]
    return 0.2125 * red + 0.7154 * green + 0.0721 * blue

rawframes = pims.Video(filename)
rawframes = as_grey(rawframes)  ## overwrite the frames files with greyscale version.

if using_darkcount:
    darkframes = pims.Video(darkcount_filename, as_grey=True)
    
if bg_flag:
    frames = rawframes # just use the rawframes as frames without any backgrounding

In [ ]:
## where to save

import datetime
today = datetime.date.today().isoformat()
#today = '2016-02-11'

myanalysisfolder = r'Research 2021\analysis'
thismovieanalysisfolder = os.path.join(myhome, 
                                 myanalysisfolder, 
                                 today,
                                 'data_taken_' + data_date + ',_movie_' + moviename)

thismovieanalysisfolder

In [ ]:
plt.imshow(rawframes[100])
plt.title('A frame from the movie')

In [ ]:
rawframes[0].shape

In [ ]:
# Take a median average over frames of the darkcount movie.
if using_darkcount and not bg_flag:
    df = np.median(darkframes,axis=0)
    plt.imshow(df)
    plt.title('Darkcount background')
    del darkframes # clear some memory
    ## the darkframe shows if any pixels on the camera are hot, or if there is unintentional light hitting the camera.

In [ ]:
if not bg_flag:
    bg = np.median(rawframes,axis=0) # may be slow
    plt.imshow(bg)
    plt.title('Brightfield background')
    ## the brightfield background shows how the illumination varies across the image

In [ ]:
if using_darkcount and not bg_flag:
    plt.imshow(bg-df)
    plt.title('Brightfield background minus darkcounts')

In [ ]:
plt.imshow(rawframes[200])
plt.title('A raw frame')

if not bg_flag:
    if using_darkcount:
        plt.imshow((rawframes[200]-df)/(bg-df))
    else:
        plt.imshow(rawframes[200]/bg)
    plt.title('A backgrounded frame')

In [ ]:
if bg_flag == True:
    print('Already backgrounded! Or you do not want me to!')
else:
    #rawframes = frames
    #del frames
    try:
        time1 = time.time()
        if using_darkcount:
            frames = (rawframes-df)/(bg-df)  # background subtract and divide (slow)
        else:
            frames = rawframes/bg # background divide
        elapsed = time.time() - time1
        bg_flag = True;
        print('Backgrounded movie in ' + str(elapsed/60.0) + ' minutes.')
        del rawframes # clear memory
    except MemoryError:
        elapsed = time.time() - time1
        frames = rawframes
        print('Unable to background divide images after ' + str(elapsed/60.0) + ' minutes; not enough memory.')

plt.imshow(frames[100])
if bg_flag == True:
    plt.title('Backgrounded frame from movie')
else:
    plt.title('Frame from movie (not backgrounded)')

In [ ]:
# find bright spots in a frame.
# featuresize must be odd. The subpixel bias will tell you if the featuresize is too big or small.
# read up on this in the trackpy literature.
# invert: True to find dark spots, False to find bright spots (make sure this is also up-to-date in the next cell)
i = 100
featuresize = 15
minmass = 100
f1 = tp.locate(frames[i], diameter=featuresize, invert=False, minmass=minmass)
tp.annotate(f1, frames[i])
tp.subpx_bias(f1)
#f1

In [ ]:
# Now that we have picked out an appropriate featuresize and settings, it's time to go through ALL the frames,
# finding the coordinates of the bright spots in each frame.

time3 = time.time()
try:
    f_coords = tp.batch(frames[:], featuresize, invert=False, minmass=minmass) # Slow!
    elapsed4 = time.time() - time3
    print('Multithreading succeeded.')
except ValueError:
    time4 = time.time()
    elapsed3 = time4-time3
    print('Failed to use multi-threading after '+ str(elapsed3/60.0) + 'min! Attempting to run on one processor!')
    f_coords = tp.batch(frames[:], featuresize, invert=False, minmass=minmass, processes=1) # Even slower!
    elapsed4 = time.time()-time4

#f_coords = pd.read_pickle('f_coords.pkl')

print('Tracked particles in ' + str(elapsed4/60.0) + ' minutes.')

# Documentation: http://soft-matter.github.io/trackpy/generated/trackpy.batch.html
# invert : Set to True if features are darker than background.

# This is an implementation of the Crocker-Grier centroid-finding algorithm.
#  Crocker, J.C., Grier, D.G. http://dx.doi.org/10.1006/jcis.1996.0217

tp.subpx_bias(f_coords)

In [ ]:
# tell me how many frames are in the movie
nframes = f_coords['frame'].max() - f_coords['frame'].min() + 1
nframes

In [ ]:
# We have just built a list of coordinates called f_coords where we have seen particles. '
# Now we want to link these together from one frame to the next 
# so we can identify the trajectory for each particle.

# Documentation: http://soft-matter.github.io/trackpy/generated/trackpy.link_df.html

t = tp.link_df(f=f_coords, search_range=10, memory=3)
#t = pd.read_pickle('t.pkl')

# search_range gives the maximum distance features can move between frames. 
#              I think it's measured in pixels.
# memory gives the maximum number of frames during which a feature can vanish, 
#        then reappear nearby, and still be considered the same particle.
# This will run faster if the numba package is available.

#trajectory = tp.plot_traj(t, superimpose = frames[500], label=False)
# plots trajectory in pixels

In [ ]:
trajectory_plot = tp.plot_traj(t, superimpose = frames[nframes/2], label=False)

In [ ]:
# only keep trajectories that last at least this many frames
t1 = tp.filter_stubs(t, 50)
# Compare the number of particles in the unfiltered and filtered data.
print('Before:', t['particle'].nunique())
print('After:', t1['particle'].nunique())

In [ ]:
tracks = t1['particle'].astype(int).unique()

print(size(tracks))

In [ ]:
trajectory_plot = tp.plot_traj(t1, superimpose = frames[nframes/2], label=False)

In [ ]:
try:
    axes().set_aspect('equal', 'datalim') 
except:
    pass
trajectory_plot = tp.plot_traj(t1, mpp=scaling)

#savefig()

In [ ]:
d = tp.compute_drift(t1, smoothing=15)

#plt.figure()
d.plot(grid=False)
plt.title('Drift in ' + moviename + '\n')

In [ ]:
tm = tp.subtract_drift(t1, d)
plt.figure()
try:
    axes().set_aspect('equal', 'datalim') 
except:
    pass
tp.plot_traj(tm, mpp=scaling)

plt.figure()
tp.plot_traj(tm, superimpose = frames[0], label=False)

In [ ]:
nframes/fps

In [ ]:
thismovieanalysisfolder

In [ ]:
im=tp.imsd(tm, mpp=scaling, fps=fps, max_lagtime=1000)

In [ ]:
fig, ax = plt.subplots()
ax.plot(im.index, im, 'k-', alpha=0.15)  # black lines, semitransparent
ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $\Delta{}t$ [s]')
ax.set_xscale('log')
ax.set_yscale('log')
fig.set_size_inches(3,3)
plt.title('MSD, drift-subtracted\n'+ moviename + '\n')

#savefig(os.path.join(thismovieanalysisfolder, movienumber + ',_drift-subtracted-MSD.pdf'))
#savefig(os.path.join(thismovieanalysisfolder, movienumber + ',_drift-subtracted-MSD.png'))

In [ ]:
MSD_no_drift=tp.imsd(t1, mpp=scaling, fps=fps, max_lagtime=1000)

In [ ]:
fig, ax = plt.subplots()
ax.plot(MSD_no_drift.index, MSD_no_drift, 'k-', alpha=0.15)  # black lines, semitransparent
ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $\Delta{}t$ [s]')
ax.set_xscale('log')
ax.set_yscale('log')
fig.set_size_inches(3,3)
plt.title('MSD, not drift-subtracted\n'+ moviename + '\n')

#savefig(os.path.join(thismovieanalysisfolder, movienumber + ',_drift-not-subtracted-MSD.pdf'))
#savefig(os.path.join(thismovieanalysisfolder, movienumber + ',_drift-not-subtracted-MSD.png'))

In [ ]:
em = tp.emsd(tm, mpp=scaling, fps=fps)

In [ ]:
ensemble_msd_drift_sub = tp.emsd(tm, scaling, fps)
ax = ensemble_msd_drift_sub.plot(loglog=True, figsize = [3,3], style='k.',  grid=False)

ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]', xlabel='lag time $\Delta{}t$ [s]')
plt.title('ensemble MSD of drift-subtracted trajectory,\n' + moviename + '\n')
tp.utils.fit_powerlaw(em)  # performs linear best fit in log space, plots

#savefig(os.path.join(thismovieanalysisfolder, movienumber + ',_drift-subtracted-ensembleMSD.pdf'))
#savefig(os.path.join(thismovieanalysisfolder, movienumber + ',_drift-subtracted-ensembleMSD.png'))

In [ ]:
# error # don't run the following code unless you intend to

In [ ]:
if not os.path.exists(thismovieanalysisfolder):
    os.makedirs(thismovieanalysisfolder)
    print('Created ' + thismovieanalysisfolder)
else:
    print ('Already exists: ' + thismovieanalysisfolder )

In [ ]:
f_coords_filename = os.path.join(thismovieanalysisfolder, 'f_coords.pkl')

f_coords.to_pickle(f_coords_filename)
f_coords.head()

In [ ]:
t_filename = os.path.join(thismovieanalysisfolder, 't.pkl')
t.to_pickle(t_filename)
t.head()


In [ ]:
t1_filename = os.path.join(thismovieanalysisfolder, 't1.pkl')
t1.to_pickle(t1_filename)
t1.head()

In [ ]:
tm_filename = os.path.join(thismovieanalysisfolder, 'tm.pkl')
tm.to_pickle(tm_filename)
tm.head()

In [ ]:
MSD_no_drift_filename = os.path.join(thismovieanalysisfolder, 'MSD_no_drift.csv')
MSD_no_drift.to_csv(MSD_no_drift_filename)
MSD_no_drift.head()

In [ ]:
im_filename = os.path.join(thismovieanalysisfolder, 'im.csv')
im.to_csv(im_filename)
im.head()

In [ ]:
em_filename = os.path.join(thismovieanalysisfolder, 'em.csv')
em.to_csv(em_filename)
em.head()

In [ ]:
# error # if uncommented, this will prevent the following code from running unless you intend it to run.

In [ ]:
#t1_filename = os.path.join(thismovieanalysisfolder, 't1.pkl')
#t1 = pd.read_pickle(t1_filename)
#t_filename = os.path.join(thismovieanalysisfolder, 't.pkl')
#t = pd.read_pickle(t_filename)
#nframes = 407

In [ ]:
trajectories_to_show = t1

#frames = rawframes     # for troubleshooting

(y_size,x_size) = frames[0].shape

nframesinmovie = nframes
#nframesinmovie = 10     # for troubleshooting

new_movie_fps = 30     # doesn't have to be the same as fps
imagesfolder = os.path.join(thismovieanalysisfolder, 'movie_traj')
os.makedirs(imagesfolder)

In [ ]:
currentmin = 1e6 ## something too large
currentmax = 0 ## something too small
for frame in frames:
    currentmin = min(frame.min(), currentmin) 
    currentmax = max(frame.max(), currentmax)
frames_min = currentmin
frames_max = currentmax

print ('Min pixel: ', frames_min)
print ('Max pixel: ', frames_max)

In [ ]:
frametime_sec = frametime/1000.0

In [ ]:
scalebar_length_um = 15
vertical_separation = 300
horizontal_separation = 50
text_bar_separation = 25
fontsz=12
approx_length_of_text_px = 135
scalebar_height = 20
scalebar_bottom = x_size -vertical_separation
scalebar_top = scalebar_bottom - scalebar_height
scalebar_length_px = scalebar_length_um/scaling
scalebar_left_px = 0 + horizontal_separation              #scalebar_right_px - scalebar_length_px
scalebar_right_px = scalebar_left_px + scalebar_length_px #y_size-horizontal_separation
scalebar_left_percent = scalebar_left_px/(x_size*1.0)
#scalebar_length_percent = scalebar_length_px/(x_size*1.0)
#xmax = scalebar_length_percent + scalebar_left_percent
xmax = scalebar_right_px /(x_size*1.0)
scalebar_greyscale = '0.1'
center_of_bar = (scalebar_left_px + scalebar_right_px)/2.0

In [ ]:
i = nframesinmovie-1

# Calculate image width and height in inches.
imagewidth=frames.shape[2]/dpi # calculate image width in inches
imageheight=imagewidth*(frames.shape[1]/frames.shape[2])
trajectorymovieframesize=[imagewidth,imageheight]

fig = plt.figure(figsize=trajectorymovieframesize,frameon=False)
ax = fig.add_axes([0,0,1,1])
thisframe = frames[i].copy()
thisframe[0][0]=frames_max
thisframe[0][1]=frames_min
ax.imshow(-thisframe,cmap='Greys')

axhspan(scalebar_top, 
            scalebar_top + scalebar_height, 
            xmin=scalebar_left_percent, 
            xmax=xmax,
            color=scalebar_greyscale, 
            alpha=0.75)
text(center_of_bar-approx_length_of_text_px/2,scalebar_top-text_bar_separation, 
         str(scalebar_length_um) + u' \u03bcm',
         fontsize=fontsz,
         color=scalebar_greyscale)
text(horizontal_separation, horizontal_separation, "{:.1f}".format(i*frametime_sec) + ' sec',
     fontsize=fontsz,
     color=scalebar_greyscale)


ylim(y_size,0)
xlim(0,x_size)
xticks([],'')
yticks([],'')

fig.dpi

In [ ]:
tracks_to_show = trajectories_to_show['particle'].astype(int).unique()

# Thanks to Becca Perry for helping me with this code.
for i in range(nframesinmovie): 
    #print(i)
    fig = plt.figure(figsize=trajectorymovieframesize,frameon=False)
    ax = fig.add_axes([0,0,1,1])
   
    implot = ax.imshow(-frames[i],
                       vmin = -frames_max, 
                       vmax = -frames_min,
                       cmap='Greys')    # Greys maps from white to black; I prefer black to white so I negate the image.
    
    traj_hist =  trajectories_to_show[trajectories_to_show.frame <=i]

    for this_particle in tracks_to_show:
        this_traj_hist = traj_hist[traj_hist.particle == this_particle]
        this_xcoords_hist = this_traj_hist['x']
        this_ycoords_hist = this_traj_hist['y']
        plot(this_xcoords_hist.values,this_ycoords_hist.values, 'r-', alpha=0.4)

    thisframefavcoords = trajectories_to_show[trajectories_to_show.frame ==i]
    xcoords = thisframefavcoords['x']
    ycoords = thisframefavcoords['y']
    plot(xcoords.values,ycoords.values, 'r.', alpha=0.2)
       
    # scalebar
    axhspan(scalebar_top, 
            scalebar_top + scalebar_height, 
            xmin=scalebar_left_percent, 
            xmax=xmax,
            color=scalebar_greyscale, 
            alpha=0.75)

    # scalebar label
    text(center_of_bar-approx_length_of_text_px/2,scalebar_top-text_bar_separation, 
         str(scalebar_length_um) + u' \u03bcm',
         fontsize=fontsz,
         color=scalebar_greyscale)
    
    # time stamp
    text(horizontal_separation, horizontal_separation, "{:.1f}".format(i*frametime_sec) + ' sec',
         fontsize=fontsz,
         color=scalebar_greyscale)
    
    ylim(y_size,0)
    xlim(0,x_size)
    xticks([],'')
    yticks([],'')
    
    savefig(os.path.join(imagesfolder, 'img' + str(i).zfill(4) +'.tif'),
            dpi = fig.dpi)
    
    if i < nframesinmovie-1:
        close()

In [ ]:
# make avi movie
os.chdir(imagesfolder)
try:
    os.system("mencoder 'mf://*.tif' -mf type=tif:fps={0} -ovc lavc -lavcopts vcodec=mpeg4:vbitrate=100000000 -oac copy -o movie.avi".format(new_movie_fps))
    if not os.path.exists('movie.avi'):
        raise Exception('Movie file not created!')